In [ ]:
import pandas as pd
import hashlib
from datetime import datetime, timedelta
import fuzzy
import re #regular expressions
import numpy as np
from Levenshtein import distance as levenshtein_distance
soundex = fuzzy.Soundex(4)

## Load Patient Data

In [ ]:
def findConfidenceLevel(first_name1, last_name1, rna_first_name1, rna_last_name1, first_name2, last_name2, rna_first_name2, rna_last_name2):
    if rna_first_name1 == rna_first_name2 and rna_last_name1 == rna_last_name2:
        return 100
    elif rna_last_name1 == rna_last_name2 and rna_first_name1[:4] == rna_first_name2[:4]:
        return 90
    elif rna_last_name1 == rna_last_name2:
        return 85
    elif rna_last_name1[0:5] == rna_last_name2[0:5] and rna_first_name1[:4] == rna_first_name2[:4]:
        return 80
    elif soundex(last_name1) == soundex(last_name2) and soundex(first_name1) == soundex(first_name2):
        return 79
    elif soundex(rna_last_name1) == soundex(rna_last_name2) and soundex(rna_first_name1) == soundex(rna_first_name2):
        return 77
    elif rna_first_name1 == rna_first_name2 and soundex(rna_last_name1[:4]) == soundex(rna_last_name2[:4]):
        return 76
    elif rna_first_name1 == rna_first_name2:
        return 60
    else:
        return 50
    
def findConfidenceLevel2(first_name1, last_name1, rna_first_name1, rna_last_name1, first_name2, last_name2, rna_first_name2, rna_last_name2):   
    if rna_first_name1 == rna_first_name2 and rna_last_name1 == rna_last_name2:
        return 100
    elif rna_last_name1 == rna_last_name2 and rna_first_name1[:4] == rna_first_name2[:4]:
        return 90
    elif rna_last_name1 == rna_last_name2 and soundex(first_name1) == soundex(first_name2):
        return 85
    elif rna_last_name1[0:5] == rna_last_name2[0:5] and rna_first_name1[:4] == rna_first_name2[:4]:
        return 80
    elif soundex(last_name1) == soundex(last_name2) and soundex(first_name1) == soundex(first_name2):
        return 79
    elif soundex(rna_last_name1) == soundex(rna_last_name2) and soundex(rna_first_name1) == soundex(rna_first_name2):
        return 77
    elif rna_first_name1 == rna_first_name2 and soundex(rna_last_name1[:4]) == soundex(rna_last_name2[:4]):
        return 76
    elif rna_last_name1 == rna_last_name2:
        return 75
    elif rna_first_name1 == rna_first_name2:
        return 60
    else:
        return 50

In [ ]:
def generate_confidence_matrix(confidence_type, confidence_func, df, threshold):
    matrix = np.zeros((df.shape[0], df.shape[0]))
    for index1, row1 in df.iterrows():
        for index2, row2 in df.iterrows():
            conf = confidence_func(
                    row1['First Name'],
                    row1['Last Name'],
                    row1['rnaFirstName'],
                    row1['rnaLastName'],
                    row2['First Name'],
                    row2['Last Name'],
                    row2['rnaFirstName'],
                    row2['rnaLastName']
            )
            if conf > threshold:
                matrix[index1][index2] = 1
            else:
                matrix[index1][index2] = 0
    return matrix

In [ ]:
def measure_accuracy(labels, df):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index1, row1 in df.iterrows():
        for index2, row2 in df.iterrows():
            if index1 >= index2:
                continue
            if row1['GroupID'] == row2['GroupID']: # P
                if labels[index1] == labels[index2]: # T
                    tp += 1
                else:
                    fn += 1
            else: # N
                if labels[index1] == labels[index2]: # F
                    fp += 1
                else: # T
                    tn += 1
    return (tp, fp, tn, fn)

In [ ]:
tp, fp, tn, fn = measure_accuracy(clustering.labels_, df_patient)

In [ ]:
(tp + tn) / (tp + fp + tn + fn)

In [ ]:
tp / (tp + fp)

In [ ]:
tp / (tp + fn)

In [ ]:
tp, fp, tn, fn

In [1]:
from data_cleaning import *

In [2]:
df = load_and_clean_data('Patient Matching Data.csv')

In [3]:
from hashing import *

In [4]:
create_hash_tokens(df)

In [5]:
df

,GroupID,PatientID,Patient Acct #,First Name,MI,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,...,Previous Zip Code,Zip Code String,National Area,Sectional Center,Delivery Area,dob_string,rnaFirstName,rnaLastName,full_name_hash,partial_name_hash
0,1,1,247028705-7,Sutton,J,Power,9/20/1945,M,1858 Sullivan Parkway,NaN,...,10557.0,93726,9,37,26,19450920,sutton,power,f513a340f370e0d1e213cbe5a7e030f3c3729928~Power,8904f84c4af7db2bba24968b9628c21a678941bb~Pow
1,1,2,NaN,Suttin,James,Power,9/21/1945,M,1859 Sullivan Parkway,#2,...,10557.0,93726,9,37,26,19450921,suttin,power,cb4eb0277fecfbdee00d5951d5e4e4f1cabbe283~Power,4e82ecca5923b548d8f4e9583e378401046302b4~Pow
2,1,3,247028705-7,Sutton,J,Power,9/20/1945,M,1858 Sullivan Parkway,NaN,...,NaN,93726,9,37,26,19450920,sutton,power,f513a340f370e0d1e213cbe5a7e030f3c3729928~Power,8904f84c4af7db2bba24968b9628c21a678941bb~Pow
3,1,4,NaN,Sutton,NaN,Power,9/20/1954,M,1858 Sullivan Parkway,NaN,...,NaN,93726,9,37,26,19540920,sutton,power,d31c28f50fed0591d910c358b3d1fa93b05f49e0~Power,944a54f27ba70e232d8ed8c541e5af2df06d2dc1~Pow
4,1,5,NaN,SUTTON,NaN,POWER,9/20/1954,M,1858 SULLIVAN PKWAY,APT 2,...,NaN,93726,9,37,26,19540920,sutton,power,63fae202660839596baf56a46c6396dbd629705b~POWER,73cdae46281d17b0df93453d065f05a419f85cb2~POW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,61,197,NaN,Bill,NaN,Smith,5/16/1972,M,16595 City View Lane,NaN,...,NaN,98684,9,86,84,19720516,bill,smith,4c04d8977da3a043584cd6190536d8f576e6b9b2~Smith,12ac8663b465bc1eabfd52456161e3587a332fc1~Smi
197,62,198,NaN,Bill,NaN,Smith,5/16/1972,M,NaN,NaN,...,NaN,00000,0,0,0,19720516,bill,smith,4c04d8977da3a043584cd6190536d8f576e6b9b2~Smith,12ac8663b465bc1eabfd52456161e3587a332fc1~Smi
198,63,199,NaN,Sarah,NaN,Field,3/17/2010,F,9850 Kelso Road,NaN,...,NaN,98626,9,86,26,20100317,sarah,field,caf30d9710cf2852506b6752a27a3c84e5180271~Field,c0d4a6a68381657e4b4ab1a7abde60cbb1358e48~Fie
199,64,200,NaN,Sara,NaN,Field,3/17/2010,F,NaN,NaN,...,NaN,00000,0,0,0,20100317,sara,field,10d69a413a0841682292498fceef1edf6498641e~Field,c0d4a6a68381657e4b4ab1a7abde60cbb1358e48~Fie
